In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, _tree
from sklearn.metrics import accuracy_score

# Load the data
data = pd.read_csv('drug200_1722406166375.csv')

# Prepare features and target
X = data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']]
y = data['Drug']

# Perform one-hot encoding for categorical variables
X = pd.get_dummies(X, columns=['Sex', 'BP', 'Cholesterol'], drop_first=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class CustomDecisionTreeClassifier(DecisionTreeClassifier):
    def __init__(self, criterion, min_purity):
        super().__init__(criterion=criterion)
        self.min_purity = min_purity

    def fit(self, X, y):
        super().fit(X, y)
        self._prune_tree()

    def _prune_tree(self):
        if not hasattr(self, 'tree_'):
            return

        def prune_node(node):
            if self.tree_.feature[node] == _tree.TREE_UNDEFINED:
                return

            left_child = self.tree_.children_left[node]
            right_child = self.tree_.children_right[node]

            if left_child == right_child:
                return

            # Recursively prune children
            prune_node(left_child)
            prune_node(right_child)

            # Check if the node should be pruned
            if self._should_prune(node):
                self.tree_.feature[node] = _tree.TREE_UNDEFINED
                self.tree_.threshold[node] = -2
                self.tree_.children_left[node] = self.tree_.children_right[node] = _tree.TREE_LEAF

        prune_node(0)  # Start pruning from the root node

    def _should_prune(self, node):
        return self._calculate_purity(node) >= self.min_purity

    def _calculate_purity(self, node):
        total_samples = np.sum(self.tree_.value[node])
        max_class_samples = np.max(self.tree_.value[node])
        return max_class_samples / total_samples if total_samples > 0 else 0

# Create and train the model
clf = CustomDecisionTreeClassifier(criterion='entropy', min_purity=0.8)
clf.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Print the decision tree
print("\nDecision Tree:")
print(export_text(clf, feature_names=list(X.columns)))

Accuracy: 1.0000

Decision Tree:
|--- Na_to_K <= 14.83
|   |--- BP_NORMAL <= 0.50
|   |   |--- BP_LOW <= 0.50
|   |   |   |--- Age <= 50.50
|   |   |   |   |--- class: drugA
|   |   |   |--- Age >  50.50
|   |   |   |   |--- class: drugB
|   |   |--- BP_LOW >  0.50
|   |   |   |--- Cholesterol_NORMAL <= 0.50
|   |   |   |   |--- class: drugC
|   |   |   |--- Cholesterol_NORMAL >  0.50
|   |   |   |   |--- class: drugX
|   |--- BP_NORMAL >  0.50
|   |   |--- class: drugX
|--- Na_to_K >  14.83
|   |--- class: drugY

